In [1]:
from sbalign.training.diffusivity import *
from torch.distributions.multivariate_normal import MultivariateNormal
import matplotlib.pyplot as plt

In [2]:
fbb = FBB(H=0.5, K=0)

a = torch.zeros(1)
b = torch.ones(1)

N=100
time = torch.linspace(0,1,N)
X = torch.zeros(N)
for i,t in enumerate(time):
    mean,cov = fbb.pinned_statistics(t,a,b)
    mvn = MultivariateNormal(mean,cov)
    X[i] = mvn.sample(sample_shape=(1,))

plt.plot(X)

torch.Size([1, 1, 1, 1, 1])
torch.Size([1, 1, 1, 1, 1])


IndexError: too many indices for tensor of dimension 0